In [80]:
import pandas as pd
import glob
import csv
from collections import Counter
import os
from datetime import datetime
!pip install pyarrow
import pyarrow.parquet as pq

הפונקציה הראשית נמצאת בסוף הדף, נית לראות בה את סדר זימון הפעולות עם הסבר.

In [75]:
#--------סעיף 2

In [63]:
#המרת קובץ EXCEL ל CSV
def convert_excel_to_csv(input_file, output_file):
    df = pd.read_excel(input_file, engine='openpyxl')  
    df.to_csv(output_file, index=False, encoding='utf-8') 

In [43]:
convert_excel_to_csv("time_series.xlsx", "time_series_csv.csv")

In [4]:
df=pd.read_csv("time_series_csv.csv")

In [6]:
df.head(6)

,timestamp,value
0,2025-06-28 12:00:52,18.5
1,2025-06-01 04:17:23,46.3
2,2025-06-10 17:02:57,76
3,2025-06-23 05:23:22,56.4
4,2025-06-05 07:20:08,67.9
5,2025-06-26 22:27:02,87


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   timestamp  1000000 non-null  object
 1   value      900960 non-null   object
dtypes: object(2)
memory usage: 15.3+ MB


In [8]:
df.describe(include='all')
#קיימים ערכים כפולים
#רק ב valur קיימים ערכי NULL 

,timestamp,value
count,1000000,900960
unique,829625,1002
top,2025-06-02 20:42:15,not_a_number
freq,6,49969


In [11]:
#פונקציה הבודקת האם הפורמט של התאריך/שעה תקין או לא
def check_format(date):
    try:
        formatDate = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return 1
    except:
        return -1
    
count = df.shape[0]
df_date = df['timestamp']
for timestamp in df_date:
    result = check_format(timestamp)
    if result < 0:
        count += result
    
print(f"num of error val = {df.shape[0] - count}")
    




num of error val = 0


In [5]:
#פונקציה הבודקת אם יש כפילויות
def duplicates(df):
    duplicated_rows = df.duplicated(keep='first')
    print(f"There are {duplicated_rows.sum()} duplicates rows")
duplicates(df)

There are 2598 duplicates rows


In [13]:
#פונקציה הבודקת אם ההחודש והיום בתאריך בטווח הנכון - אם התאריך תקין
def check_date(date):
    try:
        date = pd.to_datetime(date)
        return 1
    except (ValueError, TypeError):
        return -1
    
count = df.shape[0]
df_date = df['timestamp']
for timestamp in df_date:
    result = check_date(timestamp)
    if result < 0:
        count += result
    
print(f"num of error date = {df.shape[0] - count}")

num of error date = 0


In [64]:
# print(df.dtypes)

#פונקציה המחשבת את הממוצע השעתי
def hourly_mean(df):
    coerced = pd.to_numeric(df['value'], errors='coerce')
    invalid_mask = coerced.isna() & df['value'].notna()
#     print("שורות עם ערכים שלא ניתנים להמרה למספר:")
#     print(df.loc[invalid_mask])
    df_clean = df.loc[~invalid_mask].copy()

    df_clean['value'] = pd.to_numeric(df_clean['value'], errors='raise')
    df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'],format='%Y-%m-%d %H:%M:%S', errors='raise') 
#     print(df.dtypes)

    df_clean['hour'] = df_clean['timestamp'].dt.floor('H')
    hourly_mean = (df_clean.groupby('hour', sort=True)['value'].mean().reset_index(name='AverageValue'))
    return hourly_mean

# hourly_mean(df)

In [65]:
#פונקציה המחלקת את הקובץ הגדול לחלקים קטנים לפי תאריך
def split_by_date(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date  # שליפת תאריך בלי השעה
    unique_dates = df['date'].unique()    # רשימת כל התאריכים הייחודיים

    for date in unique_dates:
        daily_df = df[df['date'] == date]
        filename = f"split_date_{date}.csv"
        daily_df.to_csv(filename, index=False, encoding="utf-8")

    days_csv_arr = glob.glob("split_date_*.csv")
    return days_csv_arr

df= pd.read_csv("time_series_csv.csv")
dates_csv_arr = split_by_date(df)
# dates_csv_arr

In [66]:
def date_file_with_mean(dates_csv_arr):
    for file in dates_csv_arr:
        df = pd.read_csv(file)
        hourly_mean(df).to_csv(f"hour_mean_{file}", index=False, encoding="utf-8")
        
date_file_with_mean(dates_csv_arr)       
hour_mean_dates_csv_arr = glob.glob("hour_mean_split_date_*.csv")
hour_mean_dates_csv_arr

['hour_mean_split_date_2025-06-01.csv',
 'hour_mean_split_date_2025-06-02.csv',
 'hour_mean_split_date_2025-06-03.csv',
 'hour_mean_split_date_2025-06-04.csv',
 'hour_mean_split_date_2025-06-05.csv',
 'hour_mean_split_date_2025-06-06.csv',
 'hour_mean_split_date_2025-06-07.csv',
 'hour_mean_split_date_2025-06-08.csv',
 'hour_mean_split_date_2025-06-09.csv',
 'hour_mean_split_date_2025-06-10.csv',
 'hour_mean_split_date_2025-06-11.csv',
 'hour_mean_split_date_2025-06-12.csv',
 'hour_mean_split_date_2025-06-13.csv',
 'hour_mean_split_date_2025-06-14.csv',
 'hour_mean_split_date_2025-06-15.csv',
 'hour_mean_split_date_2025-06-16.csv',
 'hour_mean_split_date_2025-06-17.csv',
 'hour_mean_split_date_2025-06-18.csv',
 'hour_mean_split_date_2025-06-19.csv',
 'hour_mean_split_date_2025-06-20.csv',
 'hour_mean_split_date_2025-06-21.csv',
 'hour_mean_split_date_2025-06-22.csv',
 'hour_mean_split_date_2025-06-23.csv',
 'hour_mean_split_date_2025-06-24.csv',
 'hour_mean_split_date_2025-06-25.csv',


In [68]:
#פונקציה הממזגת את כל הקבצים עם הממוצעים לקובץ אחד גדול
def merge_hourly_avg_files(all_files):
    dfs = []
    for file in all_files:
        df = pd.read_csv(file)
        dfs.append(df)

    final_df = pd.concat(dfs)
    final_df.to_csv("final_hourly_averages.csv", index=False, encoding='utf-8-sig')

    return final_df

final_df = merge_hourly_avg_files(hour_mean_dates_csv_arr)
final_df

,hour,AverageValue
0,2025-06-01 00:00:00,50.562894
1,2025-06-01 01:00:00,49.939803
2,2025-06-01 02:00:00,49.457213
3,2025-06-01 03:00:00,50.181573
4,2025-06-01 04:00:00,48.611496
...,...,...
19,2025-06-30 19:00:00,50.907897
20,2025-06-30 20:00:00,48.924813
21,2025-06-30 21:00:00,50.754128
22,2025-06-30 22:00:00,48.178125


סעיף 2.3

במקרה שהקבצים מגיעים בזרימה והייתי צריכה לחשב ממוצע שעתי בזמן אמת, אלו היו השינויים שהייתי עושה:

לקובץ הסופי בו יש את הממוצעים השעתיים הייתי מוסיפה עוד 2 עמודות:

1.sum-עמודה שתשמור את הסכום של כל ערכי הtimestamp שנכנסו באותה שעה ובאותו תאריך.

2.count- עמודה שתשמור את כמות הפעמים שנכנסו timestamp באותה שעה ובאותו תאריך.

כשיש לי את העמודות האלו אני יכולה לעדכן אותם נקודתית ובצורה מהירה בכל כניסה חדשה שהוזרמה ולפיהן גם לעדכן את הממוצע בקלות וביעילות.

In [76]:
#פונקציה המטעינה את הדאטה בהתאם לפורמט של הקובץ
def load_file(file_path):
    ext = os.path.splitext(file_path)[-1]
    if ext ==".xlsx":
        convert_excel_to_csv(file_path, "time_series_csv.csv")
        return pd.read_csv("time_series_csv.csv")
    elif ext == ".csv":
        return pd.read_csv(file_path)
    elif ext == ".parquet":
        return pq.read_table(file_path).to_pandas()
    else:
        raise ValueError("Unsupported file type")

In [84]:
#פונקציה כללית ראשית הנעזרת בפונקציות שלעיל הכוללת תמיכה בקובץ .parquet
#קלט קובץ
#פלט- קובץ המכיל ממוצע שעתי

def mean_hours(file):
    #טעינת הדאטה
    df = load_file(file)
    
    #זימון הפונקציה שמחלקת את הקובץ לחלקים קטנים
    dates_csv_arr = split_by_date(df)
    
    #זימון הפונקציה שיוצרת לכל חלק קובץ עם ממוצע שעתי
    date_file_with_mean(dates_csv_arr)       
    hour_mean_dates_csv_arr = glob.glob("hour_mean_split_date_*.csv")
    
    #זימון הפונקציה הממזגת את כל החלקים עם הממוצעים לקובץ אחד
    final_df = merge_hourly_avg_files(hour_mean_dates_csv_arr)
    
    ext = os.path.splitext(file)[-1]
    if ext == ".parquet":
        final_df.to_parquet("final_hourly_averages.parquet", index=False)
    
    return final_df

mean_hours("time_series.xlsx") 
# mean_hours("time_series (4) (2).parquet")

,hour,AverageValue
0,2025-06-01 00:00:00,50.562894
1,2025-06-01 01:00:00,49.939803
2,2025-06-01 02:00:00,49.457213
3,2025-06-01 03:00:00,50.181573
4,2025-06-01 04:00:00,48.611496
...,...,...
19,2025-06-30 19:00:00,50.907897
20,2025-06-30 20:00:00,48.924813
21,2025-06-30 21:00:00,50.754128
22,2025-06-30 22:00:00,48.178125


In [83]:
df = load_file("time_series (4) (2).parquet")
print(df.columns)


Index(['timestamp', 'mean_value', 'median_value', 'std_dev', 'count'], dtype='object')


יתרונות של parquet:

1. אחסון יעיל עם פורמט עמודות

2. דחיסה מעולה - תופס פחות זיכרון

3. מותאם לעומסי עבודה ביג דאטה

4. ביצועי שאילתות מהירים יותר

5. סכמה מובנית ומטא נתונים

6. תמיכה בנתונים מורכבים ומקוננים

7. תאימות בין פלטפורמות ומערכות אקולוגיות

8. יעילות עלות